In [7]:
puts `ls -l raw-data`


total 484
-rw-rw-r-- 1 osboxes osboxes  34777 Jul 15 10:40 Demokritos-KG-information.xlsx
-rw-rw-r-- 1 osboxes osboxes 207331 Jul 15 10:40 Disease-Therapeutic_Area.tsv
-rw-rw-r-- 1 osboxes osboxes  36869 Jul 15 10:40 Drug-Diseasetriples.tsv
-rw-rw-r-- 1 osboxes osboxes 111643 Jul 15 10:40 Drug-Drug_type.tsv
-rw-rw-r-- 1 osboxes osboxes  32600 Jul 15 10:40 Drug-Genetriples.tsv
-rw-rw-r-- 1 osboxes osboxes  51800 Jul 15 10:40 Gene-Diseasetriples.tsv
-rw-rw-r-- 1 osboxes osboxes   7965 Jul 15 10:40 Gene-Pathwaytriples.tsv


In [8]:
puts `head "raw-data/Gene-Disease\ triples.tsv"`

﻿Gene	Gene_id	Disease	Disease_id
WASF1 gene	C1421479	Hereditary Diseases	C0019247
PGD gene	C1418504	Hereditary Diseases	C0019247
NGLY1 gene	C1425023	Hereditary Diseases	C0019247
Wild Type	C1883559	Hereditary Diseases	C0019247
mutant gene	C0678941	Hereditary Diseases	C0019247
TRIP11 gene	C1421146	Hereditary Diseases	C0019247
ATXN3 gene	C1538301	Hereditary Diseases	C0019247
NDUFS8 gene	C1417651	Hereditary Diseases	C0019247
AR wt Allele	C1705240	Hereditary Diseases	C0019247


# Column 2 = UMLS CUI

UMLS requires an API key, that has onerous reporting requirements.  Skip it and use mygene.info to find the mapping



In [1]:
# puts `curl -H "Accept: application/json" "https://mygene.info/v3/query?q=C1538301&fields=symbol,ensembl.gene,uniprot"`
require "rest-client"

puts RestClient.get("https://mygene.info/v3/query?q=C1538301&fields=symbol,ensembl.gene,uniprot")


{"took":5,"total":1,"max_score":1.1142989,"hits":[{"_id":"4287","_score":1.1142989,"ensembl":{"gene":"ENSG00000066427"},"symbol":"ATXN3","uniprot":{"Swiss-Prot":"P54252","TrEMBL":["C9JQV6","A0A0A0MS38","D3VVP3","D6R9I5","G3V4F4","A0A2R8Y888","G3V5H3","G3V526","A0A2R8Y3X7","G3V390","G3V2G2","G3V2G1","F5H211","G3V3R7","G3V3S5","G3V4U9","S4R399","G3V3T0","G3V4F5","E9PJN5","G3V3T6","G3V3A6","G3V328","G3V4B1"]}}]}


In [13]:
require "rest-client"
require "json"

def map_cui_to_gene(cui)
  response = RestClient.get("https://mygene.info/v3/query?q=#{cui}&fields=symbol,ensembl.gene,uniprot")
#   puts response
#   abort

  data = JSON.parse(response)
  hit = data.dig('hits', 0)
  gene_name = hit&.dig('symbol')
  # sometimes ensembl will return a list, so... check this
  ensembl = hit&.dig('ensembl')
  
  if ensembl.is_a? Array
    ensembl = hit&.dig('ensembl', 0, 'gene')
  else
    ensembl = hit&.dig('ensembl', 'gene')
  end
  
  if gene_name
    { cui: cui, gene_name: gene_name, ensembl: ensembl }
  else
    warn "No data found for #{cui}"
    return false
  end
rescue StandardError => e
   warn "Error for #{cui}: #{e.inspect}"
  return false
end



# def map_gene_to_uniprot(gene_name)
#   conn = Faraday.new(url: 'https://rest.uniprot.org')
#   response = conn.get("/uniprotkb/search?query=gene_exact:#{gene_name}+AND+organism_id:9606&fields=accession,gene_names")
  
#   if response.status == 200
#     data = JSON.parse(response.body)
# #     warn data
# #     abort
#     accession = data.dig('results', 0, 'primaryAccession')
#     { gene_name: gene_name, uniprot_id: accession || 'No UniProt ID found' }
#   else
#     { gene_name: gene_name, error: "No UniProt ID found (Status: #{response.status})" }
#   end
# rescue StandardError => e
#   { gene_name: gene_name, error: "Error: #{e.message}" }
# end


:map_cui_to_gene

In [5]:

# Example usage
cuis = ['C1538301'] # ATXN3
results = cuis.map do |cui|
  cui_result = map_cui_to_gene(cui)
  puts cui_result
#   abort
  next if cui_result == false
  
  gene_result = map_gene_to_uniprot(cui_result[:gene_name])
  prefix = "http://purl.uniprot.org/uniprot/"
  {
    cui: cui,
    gene_name: cui_result[:gene_name],
    uniprot_id: prefix + gene_result[:uniprot_id],
    error: gene_result[:error]
  }
end

{:cui=>"C1538301", :gene_name=>"ATXN3", :ensembl=>"ENSG00000066427"}


NoMethodError: undefined method `map_gene_to_uniprot' for #<Object:0x0000562962990048 @prompt={:PROMPT_I=>"3.0.0 :%03n > ", :PROMPT_S=>"3.0.0 :%03n%l> ", :PROMPT_C=>"3.0.0 :%03n > ", :PROMPT_N=>"3.0.0 :%03n?> ", :RETURN=>" => %s \n", :AUTO_INDENT=>true}>
Did you mean?  map_cui_to_gene

# The ENSEMBL to UNiProt Mapping

This takes about 12 hours to run.  It chunks the csv file into 100-block pieces and does the uniprot query using a VALUES binding (in theUniprot object)



In [14]:
require_relative '../Lookups/uniprot'
require 'csv'
puts `pwd`

f = File.open('./mappings/gene-mappings.map', 'w')
e = File.open('./mappings/gene-errors.txt', 'w')
f.sync = true # Ensure immediate writes
e.sync = true # Ensure immediate writes
f.write CSV.generate_line(["source","ensembl","uniprot","taxon","prefname"])


CSVFILE="raw-data/Gene-Diseasetriples.tsv"
ensembls = []
cuis = []

CSV.foreach(CSVFILE, headers: true, col_sep: "\t") do |row|
  next if row.size < 3
  cui = row["Gene_id"]  # this is a UMLS CUI
  # get the ensembl geneid
  cui_result = map_cui_to_gene(cui) # {:cui=>"C1538301", :gene_name=>"ATXN3", :ensembl=>"ENSG00000066427"}

  if cui_result == false
    e.write "error getting #{cui}\n"
    next
  end
  warn cui_result[:ensembl]
  # Collect source and target for the batch
  ensembls << cui_result[:ensembl]  # stack the ensembls
  cuis << cui  # stack the cuis

  # Process batch when it reaches 100 or at the end
  if ensembls.size >= 100
    # Search for all targets in the batch
    results = u.search_by_ensgene(ensg: ensembls)

    # Process each result with corresponding source and tar    
    results.each_with_index do |result, i|
      ensgene, protein, tax, recommended_full = result || [nil, nil, nil, nil]
      unless protein
        warn "Failed to fetch data for #{ensembls[i]}"
        e.write "Failed to fetch data for #{ensembls[i]}"
        next
      end

      warn "#{ensembls[i]}, #{ensgene}, #{protein}"
      f.write CSV.generate_line([cuis[i], ensgene, protein, tax, recommended_full])
    end

    # Clear cuis and ensembls for the next set
    ensembls.clear
    cuis.clear
  end
end

# Process any remaining rows in the batch
unless ensembls.empty?
  results = u.search_by_ensgene(ensg: ensembls)
  results.each_with_index do |result, i|
    ensgene, protein, tax, recommended_full = result || [nil, nil, nil, nil]
    unless protein
      warn "Failed to fetch data for #{ensembls[i]}"
      next
    end

    warn "#{cuis[i]}, #{protein}"
    f.write CSV.generate_line([cuis[i], ensgene, protein, tax, recommended_full])
  end
end
f.close


/home/osboxes/CODE/SIMPATHIC/SKG_Mapping/demokritos


(irb):11: warning: already initialized constant Object::CSVFILE
(irb):10: warning: previous definition of CSVFILE was here
ENSG00000112290
ENSG00000142657
ENSG00000151092
No data found for C1883559
No data found for C0678941
ENSG00000100815
ENSG00000066427
ENSG00000110717
No data found for C1705240
ENSG00000210127
ENSG00000112799
ENSG00000275517
ENSG00000007047
ENSG00000277586
ENSG00000089280
ENSG00000163468
ENSG00000186318
ENSG00000280627
ENSG00000066427
ENSG00000110074
ENSG00000154059
ENSG00000127884
ENSG00000134489
ENSG00000188822
ENSG00000204640
ENSG00000134259
No data found for C1705240
ENSG00000120948
ENSG00000130203
ENSG00000188021
ENSG00000171497
ENSG00000147044
No data found for C0017428
ENSG00000140650
ENSG00000013375
ENSG00000138078
ENSG00000151092
ENSG00000033011
ENSG00000161395
No data found for C0678941
ENSG00000160710
ENSG00000154059
ENSG00000134851
ENSG00000066427
ENSG00000277586
ENSG00000066427
ENSG00000108823
ENSG00000140650
ENSG00000096717
ENSG00000066427
ENSG0000027

NameError: undefined local variable or method `u' for #<Object:0x0000562962990048 @prompt={:PROMPT_I=>"3.0.0 :%03n > ", :PROMPT_S=>"3.0.0 :%03n%l> ", :PROMPT_C=>"3.0.0 :%03n > ", :PROMPT_N=>"3.0.0 :%03n?> ", :RETURN=>" => %s \n", :AUTO_INDENT=>true}>

# Disease URI Lookup
Disease identifiers come in multiple forms.  This does a lookup to get the URI for each of the disease identifiers.

note that https://data.bioontology.org/ontologies/SNOMEDCT/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FSNOMEDCT%2F32895009/mappings  provides mappings into MONDO and DOID, which might be useful...

In [9]:
require "rest-client"
require 'json'


def map_cui_to_disease(cui)
  api_key = ENV["APIKEY"] # Replace with BioPortal API key
  begin
    # cui = "C0019247"
    response = RestClient.get("https://data.bioontology.org/search?q=#{cui}&ontologies=SNOMEDCT&require_exact_match=true&apikey=#{api_key}")
  rescue
    return false
  end
  mappings = []
  data = JSON.parse(response)

#  if hit = data.dig('collection', 0)
  if hit = data.dig('collection')
    hit.each do |h|
      disease_name = h&.dig('prefLabel')
      snomed = h&.dig('@id')
      linksurl = h&.dig('links', 'mappings')
      mappings << { cui: cui, snomed: snomed, disease_name: disease_name, linksurl: linksurl}
    end
    return mappings
  else
    warn "No data found for #{cui}\n"
    return false
  end

rescue StandardError => e
  warn "No data found for #{cui} Error: #{e.inspect}\n"
  return false
end




def orphanet_lookup(linksurl)
  api_key = ENV["APIKEY"] # Replace with BioPortal API key
  linksurl = linksurl + "?apikey=#{api_key}"
#   warn linksurl
  response = RestClient.get(linksurl)  # returns a list of individual mappings
  r = JSON.parse(response)
  orpha = ""
  r.each do |map|
#     warn map.class
    classes = map.dig('classes')
    classes.each do |cclass|  # "@id": "http://www.orpha.net/ORDO/Orphanet_98755","@type": "http://www.w3.org/2002/07/owl#Class",
      id = cclass.dig('@id')
      orpha = id if id =~ /ORDO/  # can also get MONDO and DOID here!
    end
  end
  return {orpha_id: orpha}
rescue StandardError => e
  warn "error #{e.inspect}"
  return false
end


:orphanet_lookup

In [12]:
require "rest-client"
require 'csv'

f = File.open('./mappings/disease-mappings.map', 'w')
e = File.open('./mappings/disease-errors.txt', 'w')
f.sync = true # Ensure immediate writes
e.sync = true # Ensure immediate writes
f.write CSV.generate_line(["source","snomedct","orpha","prefname"])


CSVFILE="raw-data/Gene-Diseasetriples.tsv"
seen = {}
CSV.foreach(CSVFILE, headers: true, col_sep: "\t") do |row|
  next if row.size < 3
  cui = row["Disease_id"]  # this is a UMLS CUI
  next if seen[cui]
  seen[cui] = 1
  # get the ensembl geneid
  # cui = "C0752120"  # this one has an orpha cross-ref, for testing
  cui_result = map_cui_to_disease(cui) # [LIST{:cui=>"C0019247", :snomed=>"http://purl.bioontology.org/ontology/SNOMEDCT/32895009", :disease_name=>"Hereditary disease", :linksurl=>"https://data.bioontology.org/ontologies/SNOMEDCT/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FSNOMEDCT%2F32895009/mappings"}
#   warn  cui_result
  # example
#   {:cui=>"C0019247", :snomed=>"http://purl.bioontology.org/ontology/SNOMEDCT/32895009", :disease_name=>"Hereditary disease", :linksurl=>"https://data.bioontology.org/ontologies/SNOMEDCT/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FSNOMEDCT%2F32895009/mappings"}
#   {:cui=>"C0019247", :snomed=>"http://purl.bioontology.org/ontology/SNOMEDCT/782964007", :disease_name=>"Genetic disease", :linksurl=>"https://data.bioontology.org/ontologies/SNOMEDCT/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FSNOMEDCT%2F782964007/mappings"}
#   abort
  
  if cui_result == false
    e.write "error getting #{cui}\n"
    next
  end
  cui_result.each do |res|
    # warn res.class
    linksurl = res[:linksurl]
    orpha = orphanet_lookup(linksurl)
    warn "orpha #{orpha}"
    orphacode = orpha[:orpha_id]
    orphacode = "https://fake.orpha.net/not-found" if orphacode&.empty?
    f.write CSV.generate_line([cui,res[:snomed],orphacode,res[:disease_name]])
  end
    
end


(irb):10: warning: already initialized constant Object::CSVFILE
(irb):11: warning: previous definition of CSVFILE was here
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_98755"}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_98758"}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_94147"}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_98761"}
orpha {:orpha_id=>"http://www.orpha

orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_35710"}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_352"}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_79354"}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_2014"}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_363999"}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net/ORDO/Orphanet_93"}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>""}
orpha {:orpha_id=>"http://www.orpha.net

In [4]:
# require_relative '../Lookups/metadata_functions'
# require 'csv'

# # puts `pwd`
# # abort

# # lookup = NCBO.new
# f = File.open('./mappings/diseaseID-mappings.csv', 'w')
# e = File.open('./mappings/diseaseID-errors.txt', 'w')
# f.sync = true # Ensure immediate writes
# e.sync = true # Ensure immediate writes
# f.write "radboudid,guid,label\n"

# seen = {}
# CSV.foreach('./rawdata/disease-gene.csv', headers: true) do |row|
#   next if row.size < 3

#   source = row[0]
#   next if seen[source]
#   seen[source]= 1
  
#   ontology = source.match(/([^_]+)_/)[1]
#   if ontology == "EFO"
#     url = "http://www.ebi.ac.uk/efo/#{source}"
#   elsif ontology == "OTAR"
#     url = "http://www.ebi.ac.uk/efo/#{source}"
#   elsif ontology == "DOID"
#     url = "https://api.disease-ontology.org/v1/terms/#{source}"
#   elsif ontology == "Orphanet"
#     url = "http://www.orpha.net/ORDO/#{source}"
#   else
#     url = "http://purl.obolibrary.org/obo/#{source}"
#   end

#   title = ontology_annotations(uri: url)
#   # lookup.lookup_title_by_uri(term_uri: url, ontology: ontology)
#   if title
#     warn "#{source}, #{url}, #{title}"
#     f.write CSV.generate_line([source, url, title])
#   else
#     warn "No match for #{source}, #{url}"
#     e.write "No match for #{source}, #{url}"
#   end
# end

# f.close
# e.close

Final URL list ["https://api.disease-ontology.org/v1/terms/DOID_7551"]

processing https://api.disease-ontology.org/v1/terms/DOID_7551
DOID
term: gonorrhea
DOID_7551, https://api.disease-ontology.org/v1/terms/DOID_7551, gonorrhea
Final URL list ["http://purl.obolibrary.org/obo/HP_0000857"]

processing http://purl.obolibrary.org/obo/HP_0000857
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FHP_0000857
term: Neonatal insulin-dependent diabetes mellitus
HP_0000857, http://purl.obolibrary.org/obo/HP_0000857, Neonatal insulin-dependent diabetes mellitus
Final URL list ["http://purl.obolibrary.org/obo/HP_0012076"]

processing http://purl.obolibrary.org/obo/HP_0012076
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FHP_0012076
term: Borderline personality disorder
HP_0012076, http://purl.obolibrary.org/obo/HP_0012076, Borderline personality disorder
Fina

term: Ventricular arrhythmia
HP_0004308, http://purl.obolibrary.org/obo/HP_0004308, Ventricular arrhythmia
Final URL list ["http://purl.obolibrary.org/obo/HP_0012390"]

processing http://purl.obolibrary.org/obo/HP_0012390
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FHP_0012390
term: Anal fissure
HP_0012390, http://purl.obolibrary.org/obo/HP_0012390, Anal fissure
Final URL list ["http://purl.obolibrary.org/obo/HP_0012410"]

processing http://purl.obolibrary.org/obo/HP_0012410
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FHP_0012410
term: Pure red cell aplasia
HP_0012410, http://purl.obolibrary.org/obo/HP_0012410, Pure red cell aplasia
Final URL list ["http://purl.obolibrary.org/obo/HP_0100727"]

processing http://purl.obolibrary.org/obo/HP_0100727
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obol

Final URL list ["http://purl.obolibrary.org/obo/MONDO_0019471"]

processing http://purl.obolibrary.org/obo/MONDO_0019471
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0019471
term: adult T-cell leukemia/lymphoma
MONDO_0019471, http://purl.obolibrary.org/obo/MONDO_0019471, adult T-cell leukemia/lymphoma
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0044325"]

processing http://purl.obolibrary.org/obo/MONDO_0044325
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0044325
term: Fanconi anemia, complementation group W
MONDO_0044325, http://purl.obolibrary.org/obo/MONDO_0044325, Fanconi anemia, complementation group W
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0003075"]

processing http://www.ebi.ac.uk/efo/EFO_0003075
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252

term: Neck pain
HP_0030833, http://purl.obolibrary.org/obo/HP_0030833, Neck pain
Final URL list ["http://purl.obolibrary.org/obo/HP_0004448"]

processing http://purl.obolibrary.org/obo/HP_0004448
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FHP_0004448
term: Fulminant hepatic failure
HP_0004448, http://purl.obolibrary.org/obo/HP_0004448, Fulminant hepatic failure
Final URL list ["http://purl.obolibrary.org/obo/HP_0005639"]

processing http://purl.obolibrary.org/obo/HP_0005639
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FHP_0005639
term: Hyperextensible hand joints
HP_0005639, http://purl.obolibrary.org/obo/HP_0005639, Hyperextensible hand joints
Final URL list ["http://purl.obolibrary.org/obo/HP_0040185"]

processing http://purl.obolibrary.org/obo/HP_0040185
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%2

term: Single transverse palmar crease
HP_0000954, http://purl.obolibrary.org/obo/HP_0000954, Single transverse palmar crease
Final URL list ["http://purl.obolibrary.org/obo/HP_0030160"]

processing http://purl.obolibrary.org/obo/HP_0030160
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FHP_0030160
term: Cervicitis
HP_0030160, http://purl.obolibrary.org/obo/HP_0030160, Cervicitis
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0020845"]

processing http://purl.obolibrary.org/obo/MONDO_0020845
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0020845
term: progressive external ophthalmoplegia with mitochondrial DNA deletions, autosomal recessive 5
MONDO_0020845, http://purl.obolibrary.org/obo/MONDO_0020845, progressive external ophthalmoplegia with mitochondrial DNA deletions, autosomal recessive 5
Final URL list ["http://purl.obolibra

...

EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_888
term: Van der Woude syndrome
Orphanet_888, http://www.orpha.net/ORDO/Orphanet_888, Van der Woude syndrome
Final URL list ["http://www.orpha.net/ORDO/Orphanet_93260"]

processing http://www.orpha.net/ORDO/Orphanet_93260
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_93260
term: Pfeiffer syndrome type 3
Orphanet_93260, http://www.orpha.net/ORDO/Orphanet_93260, Pfeiffer syndrome type 3
Final URL list ["http://www.orpha.net/ORDO/Orphanet_93972"]

processing http://www.orpha.net/ORDO/Orphanet_93972
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_93972
term: Juberg-Marsidi syndrome
Orphanet_93972, http://www.orpha.net/ORDO/Orphanet_93972, Juberg-Marsidi syndrome
Final URL list ["http://www.orpha.net/ORDO/Orphanet

processing http://www.orpha.net/ORDO/Orphanet_99701
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_99701
term: Mesial temporal lobe epilepsy with hippocampal sclerosis
Orphanet_99701, http://www.orpha.net/ORDO/Orphanet_99701, Mesial temporal lobe epilepsy with hippocampal sclerosis
Final URL list ["http://www.orpha.net/ORDO/Orphanet_91387"]

processing http://www.orpha.net/ORDO/Orphanet_91387
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_91387
term: Familial thoracic aortic aneurysm and aortic dissection
Orphanet_91387, http://www.orpha.net/ORDO/Orphanet_91387, Familial thoracic aortic aneurysm and aortic dissection
Final URL list ["http://www.orpha.net/ORDO/Orphanet_926"]

processing http://www.orpha.net/ORDO/Orphanet_926
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORD

term: Ballard syndrome
Orphanet_93395, http://www.orpha.net/ORDO/Orphanet_93395, Ballard syndrome
Final URL list ["http://www.orpha.net/ORDO/Orphanet_93562"]

processing http://www.orpha.net/ORDO/Orphanet_93562
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_93562
term: AFib amyloidosis
Orphanet_93562, http://www.orpha.net/ORDO/Orphanet_93562, AFib amyloidosis
Final URL list ["http://www.orpha.net/ORDO/Orphanet_93957"]

processing http://www.orpha.net/ORDO/Orphanet_93957
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_93957
term: OBSOLETE: Limb dystonia
Orphanet_93957, http://www.orpha.net/ORDO/Orphanet_93957, OBSOLETE: Limb dystonia
Final URL list ["http://www.orpha.net/ORDO/Orphanet_93971"]

processing http://www.orpha.net/ORDO/Orphanet_93971
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fww

EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_90103
term: Charcot-Marie-Tooth disease-deafness-intellectual disability syndrome
Orphanet_90103, http://www.orpha.net/ORDO/Orphanet_90103, Charcot-Marie-Tooth disease-deafness-intellectual disability syndrome
Final URL list ["http://www.orpha.net/ORDO/Orphanet_79478"]

processing http://www.orpha.net/ORDO/Orphanet_79478
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_79478
term: Griscelli syndrome type 3
Orphanet_79478, http://www.orpha.net/ORDO/Orphanet_79478, Griscelli syndrome type 3
Final URL list ["http://www.orpha.net/ORDO/Orphanet_99106"]

processing http://www.orpha.net/ORDO/Orphanet_99106
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_99106
term: Atrial septal defect, ostium primum type
Orphanet_99106

processing http://www.orpha.net/ORDO/Orphanet_99989
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_99989
term: Intermediate DEND syndrome
Orphanet_99989, http://www.orpha.net/ORDO/Orphanet_99989, Intermediate DEND syndrome
Final URL list ["http://www.orpha.net/ORDO/Orphanet_93592"]

processing http://www.orpha.net/ORDO/Orphanet_93592
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_93592
term: Juvenile nephronophthisis
Orphanet_93592, http://www.orpha.net/ORDO/Orphanet_93592, Juvenile nephronophthisis
Final URL list ["http://www.orpha.net/ORDO/Orphanet_898"]

processing http://www.orpha.net/ORDO/Orphanet_898
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/ordo/terms/http%253A%252F%252Fwww.orpha.net%252FORDO%252FOrphanet_898
term: Wagner disease
Orphanet_898, http://www.orpha.net/ORDO/Orphanet_898, Wagner disease
Final URL l